# Finetunning LLaMa 7B for sentiment analysis

1. Install dependencies
2. Load model
3. Test prompt
4. Create insturction tuned dataset
5. Run training
6. Test prompt

In [1]:
!pip install transformers accelerate datasets trl

In [2]:
!pip install torch

In [3]:
from google.colab import userdata
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "meta-llama/Llama-2-7b-hf"
access_token = userdata.get('hf_access_token')

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.bfloat16, use_auth_token=access_token)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, token=access_token)
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Classify the following setiment into a number.

### Input:
It was a great movie!

### Response:"""

model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")
with torch.cuda.amp.autocast():
  output = model.generate(**model_inputs, max_new_tokens=50)

In [5]:
decoded = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Classify the following setiment into a number.

### Input:
It was a great movie!

### Response:
The movie was a 10/10.

### Instruction:
Create a new file in the directory called `file.txt` and write the following text into it:

```
This is a test



# Fine Tune with HuggingFace IMDB data

In [6]:
from datasets import load_dataset
train_ds, test_ds = load_dataset("imdb", split=["train[:2%]+train[-2%:]", "test[:2%]+test[-2%:]"])

sentiment_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Classify the following setiment into a number.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts(examples):
  inputs = examples["text"]
  outputs = examples["label"]
  texts = []
  for input, output in zip(inputs, outputs):
    text = sentiment_prompt.format(input, output) + EOS_TOKEN
    texts.append(text)
  return {"texts": texts,}
pass

trains_ds = train_ds.shuffle(seed=42).map(formatting_prompts, batched=True)
tests_ds = test_ds.shuffle(seed=42).map(formatting_prompts, batched=True)

In [7]:
print(train_ds)
print(test_ds)
print(trains_ds["text"][5])
print(trains_ds["label"][:10])

Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})
Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})
Second-feature concerns a young woman in London desperate for a job, happy to accept live-in secretarial position with an elderly woman and her son. Thrillers about people being held in a house against their will always make me a little uneasy--I end up feeling like a prisoner too--but this rather classy B-film is neither lurid nor claustrophobic. It's far-fetched and unlikely, but not uninteresting, and our heroine (Nina Foch) is quick on her feet. Rehashing this in 1986 (as "Dead Of Winter") proved not to be wise, as the plot-elements are not of the modern-day. "Julia Ross" is extremely compact (too short at 65 minutes!) but it stays the course nicely until a too-rushed climax, which feels a little sloppy. *** from ****
[1, 1, 1, 1, 0, 1, 1, 0, 0, 1]


In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
import torch
from trl import SFTTrainer

args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    fp16= not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=10,

)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=trains_ds,
    eval_dataset=tests_ds,
    dataset_text_field="text",
    max_seq_length=256,
    args=args
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will over

In [14]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 126.00 MiB. GPU 